양자화 하는 파일

In [1]:
import onnx
import onnxruntime
from onnxruntime.quantization import QuantType, quantize_static, CalibrationDataReader, QuantFormat
from onnxruntime.quantization.shape_inference import quant_pre_process

import funs

import os

import pandas as pd 
import numpy as np  

In [2]:
#### Name Parameters ####

model_fp32_name = 'FRFconv-TDS_onnx'
model_fp32 = f'quant_models/{model_fp32_name}.onnx'

# 전처리된 모델
model_fp32_prep_name = "FRFconv-TDS_onnx_prep"
model_fp32_prep = f'quant_models/{model_fp32_prep_name}.onnx'

# 양자화된 모델
model_quant_name = f'{model_fp32_name}v2.quant'
model_quant = f'quant_models/{model_quant_name}.onnx'


### 기타 파라미터 ###
config = funs.load_yaml('./config.yaml')

In [3]:
def check_model(model_path):
    onnx.checker.check_model(model_path)
    print(f"{model_path} is valid!")

def preprocess_model(model_path, model_prep_path):
    quant_pre_process(model_path, model_prep_path)
    print(f"Preprocessed model saved to {model_prep_path}")

In [4]:
# 원본 모델 확인
check_model(model_fp32)

# 모델 전처리 및 확인
preprocess_model(model_fp32, model_fp32_prep)
check_model(model_fp32_prep)

quant_models/FRFconv-TDS_onnx.onnx is valid!
Preprocessed model saved to quant_models/FRFconv-TDS_onnx_prep.onnx
quant_models/FRFconv-TDS_onnx_prep.onnx is valid!


In [ ]:
# Calibration dataset 제작
funs.set_seed(config.seed)

data_root_dirs = os.path.join(config.dataset_root)

Cylindrical_dirs = funs.get_bearing_paths(data_root_dirs, 'CylindricalRoller', config.rpm, config.sampling_rate)
DepGroove_dirs = funs.get_bearing_paths(data_root_dirs, 'DeepGrooveBall', config.rpm, config.sampling_rate)
Tapered_dirs = funs.get_bearing_paths(data_root_dirs, 'TaperedRoller', config.rpm, config.sampling_rate)

print("Making dataframes...")

Cylindrical_df = funs.make_dataframe(config, Cylindrical_dirs)
DepGroove_df =funs.make_dataframe(config, DepGroove_dirs)
Tapered_df = funs.make_dataframe(config, Tapered_dirs)  

print("concat dataframes...")
all_df = pd.concat([Cylindrical_df, DepGroove_df, Tapered_df], ignore_index=True)

train_df, val_df, test_df = funs.split_dataframe(all_df, 0.6, 0.2)

# cali_data, cali_label = funs.build_from_dataframe(val_df, config.sample_size, config.overlap, False)
# cali_data = cali_data.astype(np.float32)

Making dataframes...
concat dataframes...


In [11]:
all_df

,data,fault_type,label,RPM,bearing_type
0,"[0.032029427224228385, 0.03612082315095396, 0....",M1,2,600,CylindricalRoller
1,"[-0.022020231216657013, -0.020676273442870437,...",U3,7,600,CylindricalRoller
2,"[0.02156911252335802, 0.021869858318890684, 0....",U1,5,600,CylindricalRoller
3,"[0.010855043557506978, 0.015619984755477575, 0...",U2,6,600,CylindricalRoller
4,"[0.07204114910488614, 0.06388341940106272, 0.0...",M2,3,600,CylindricalRoller
...,...,...,...,...,...
139,"[0.0962825133323, 0.3015979086200034, -0.09096...",H,0,1600,TaperedRoller
140,"[0.16041655422963993, -0.076828019683781, 0.28...",M3,4,1600,TaperedRoller
141,"[-0.06439092793102408, 0.16383753765382394, 0....",U3,7,1600,TaperedRoller
142,"[-0.2290774459034871, -0.37063159976956295, -0...",U1,5,1600,TaperedRoller


In [ ]:
import pandas as pd
import numpy as np

for fault_type, group_df in all_df.groupby('fault_type'):
    # data 컬럼을 2D array로 변환 (샘플 수, 데이터 길이)
    data_arr = np.stack(group_df['data'].to_list())  # shape: (N, L)
    # DataFrame으로 변환 (각 값이 한 컬럼)
    data_df = pd.DataFrame(data_arr)
    data_df.to_csv(f"./data/{fault_type}_data_full.csv", index=False)
    print(f"Saved {fault_type}_data_full.csv, shape={data_arr.shape}")

Saved H_data_full.csv, shape=(18, 1280000)
Saved L_data_full.csv, shape=(18, 1280000)
Saved M1_data_full.csv, shape=(18, 1280000)


In [9]:
all_df['fault_type'].value_counts()

M1    18
U3    18
U1    18
U2    18
M2    18
M3    18
H     18
L     18
Name: fault_type, dtype: int64

In [ ]:
# fault_type별로 데이터프레임을 나눠 각각 csv로 저장
for fault_type, group_df in all_df.groupby('fault_type'):
    filename = f"{fault_type}.csv"
    group_df.to_csv(fi lename, index=False)
    print(f"Saved {filename}")

Saved H.csv
Saved L.csv
Saved M1.csv
Saved M2.csv
Saved M3.csv
Saved U1.csv
Saved U2.csv
Saved U3.csv


In [7]:
# fault_type별로 데이터프레임을 나눠 각각 csv로 저장
for fault_type, group_df in all_df.groupby('fault_type'):
    filename = f"{fault_type}.csv"
    group_df.to_csv(filename, index=False)
    print(f"Saved {filename}")

Saved H.csv
Saved L.csv
Saved M1.csv
Saved M2.csv
Saved M3.csv
Saved U1.csv
Saved U2.csv
Saved U3.csv


In [6]:
class MyCalibrationDataReader(CalibrationDataReader):
    def __init__(self, data, model_path):
        self.enum_data = None
        self.data = data 

        # Use inference session to get input shape.
        session = onnxruntime.InferenceSession(model_path, None)
        batch_size, channel, length = session.get_inputs()[0].shape
        self.input_name = session.get_inputs()[0].name
        self.datasize = len(data)

    def get_next(self):
        if self.enum_data is None:
            self.enum_data = iter([
                {self.input_name: sample[np.newaxis, np.newaxis, :].astype(np.float32)}  # (2048,) → (1, 1, 2048)
                for sample in self.data
            ])
        return next(self.enum_data, None)
    
    def rewind(self):
        self.enum_data = None  # Reset the enumeration of calibration data


In [7]:
dr = MyCalibrationDataReader(cali_data, model_fp32_prep)

In [8]:
quantize_static(
    model_fp32_prep,
    model_quant,
    dr,
    quant_format=QuantFormat.QDQ,
    per_channel=False,
    weight_type=QuantType.QInt8, 
    activation_type=QuantType.QInt8, 
    reduce_range=False,
    extra_options={'WeightSymmetric': True, 'ActivationSymmetric': False},
)

In [9]:
check_model(model_quant)

quant_models/FRFconv-TDS_onnxv2.quant.onnx is valid!


In [10]:
original_model = onnx.load(model_fp32)
# 양자화 모델  
quant_model = onnx.load(model_quant)

def get_weights_size(model):
    total_size = 0
    for initializer in model.graph.initializer:
        # 실제 가중치 데이터 크기
        total_size += len(initializer.raw_data) if initializer.raw_data else 0
    return total_size

print(f"원본 가중치 크기: {get_weights_size(original_model)} bytes")
print(f"양자화 가중치 크기: {get_weights_size(quant_model)} bytes")

원본 가중치 크기: 35296 bytes
양자화 가중치 크기: 9344 bytes
